# Finding Submarines

### The problem:

*An enemy submarine emitting an unknown frequency is known to be traveling in the Puget Sound. The goal is to locate and track the submarine's location precisely over 24 hours by analyzing acoustic pressure data sampled from the Sound.*

### The solution:

The submarine's frequency signature was identified by applying the Fourier transform method to the signal samples and then examining the time average to find the dominating frequency. Once known, a Gaussian filter was applied to remove non dominating frequencies from the transform to remove noise. The signal was then recovered with the inverse Fourier
transform method allowing the submarine’s location to be tracked precisely throughout the
24 hour period.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

# Load data and reshape
data_path = pooch.retrieve(url="doi:10.5281/zenodo.13119547/subdata.npy", known_hash=None)

d = np.load(data_path)
N_grid = 64
t_scale = 49
signal = np.reshape(d, (N_grid, N_grid, N_grid, 49))

In [ ]:
# MODIFICATION: for np.fft.fftn in this case we have to specify the axes
# we're applying the transform across all three dimensions
f_hat_sig = np.fft.fftshift(np.fft.fftn(signal, axes = (0,1,2)))
avg_f_hat_sig = np.mean(f_hat_sig, 3)

# find the maximum value and its corresponding 3D indices
max_index = np.argmax(avg_f_hat_sig)
max_indices = np.unravel_index(max_index, avg_f_hat_sig.shape)

# this yields the same result as averaging before the fourier transform
print(np.max(avg_f_hat_sig))
print(max_indices)

In [ ]:
# Set up 3D Grids

axis = np.linspace(-N_grid/2, N_grid/2, N_grid)

X, Y, Z = np.meshgrid(axis, axis, axis)

In [ ]:
# When we plot, we see a distinct frequency signature in 3D space

dat = avg_f_hat_sig

normal_abs_dat = np.abs(dat)/np.abs(dat).max()
fig_data = go.Isosurface(x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
                        value=normal_abs_dat.flatten(), isomin=0.5, isomax=1)
fig = go.Figure(data=fig_data)
fig.show()

In [ ]:
# 3D Gaussian Function

def gaussian_3d(x, y, z, mu_x, mu_y, mu_z, sigma):
    return np.exp(-((x-mu_x)**2 + (y-mu_y)**2 + (z-mu_z)**2) / (sigma**2))

In [ ]:
# Get Gaussian filter centered over central frequency

sigma = 10
gauss_filter = gaussian_3d(X, Y, Z, X[max_indices], Y[max_indices], Z[max_indices], sigma)

In [ ]:
# Choose a time sample
# Fourier transform the sample 
# Apply filter and go back to signal domain
sample = signal[:,:,:,37]
f_hat = np.fft.fftshift(np.fft.fftn(sample))
clean_signal = np.fft.ifftn(np.fft.ifftshift(f_hat*gauss_filter))

In [ ]:
# Plot the sample

dat = gauss_filter

normal_abs_dat = np.abs(dat)/np.abs(dat).max()
fig_data = go.Isosurface(x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
                        value=normal_abs_dat.flatten(), isomin=0.6, isomax=1)
fig = go.Figure(data=fig_data)
fig.show()

In [ ]:
# Get the X,Y coordinates of the submarine at each time sample

points = []
for t in range(0, t_scale):
    # For each sample get X, Y coords of maximum value
    s = signal[:,:,:,t]
    s_hat = np.fft.fftshift(np.fft.fftn(s))
    clean_s = np.fft.ifftn(np.fft.ifftshift(s_hat*gauss_filter))
    max_index = np.argmax(clean_s)
    max_indices = np.unravel_index(max_index, s_hat.shape)
    points.append(max_indices[:-1])



In [ ]:
# Plot the submarine's location over time
x_list, y_list = zip(*points)

# add color bar for clarity
t = range(0, 49, 1)

plt.scatter(x_list, y_list, c = t, marker='o')
plt.xlabel("X")
plt.ylabel("Y")
plt.xlim(0, 64)
plt.ylim(0, 64)
plt.locator_params(axis='x', nbins=13)
plt.locator_params(axis='y', nbins=13)
plt.grid(True)
plt.colorbar(label = "Time Entry")
plt.title("X,Y View of Submarine Path")
plt.show()

In [ ]:
# Plot a 3D view of the Submarine's location for each time frame

# import libraries for plotting isosurfaces
import plotly
import plotly.graph_objs as go
# utility for clearing output of cell as loop runs in notebook
from IPython.display import clear_output

# plot the data in time 

L = 10; # length of spatial domain (cube of side L = 2*10)
N_grid = 64; # number of grid points/Fourier modes in each direction
xx = np.linspace(-L, L, N_grid+1) #spatial grid in x dir
x = xx[0:N_grid]
y = x # same grid in y,z direction
z = x

K_grid = (2*np.pi/(2*L))*np.linspace(-N_grid/2, N_grid/2 -1, N_grid) # frequency grid for one coordinate

xv, yv, zv = np.meshgrid( x, y, z) # generate 3D meshgrid for plotting

# plot iso surfaces for every third measurement

for j in range(0,49,3):

  signal = np.reshape(d[:, j], (N_grid, N_grid, N_grid))
  signal = np.fft.fftshift(np.fft.fftn(signal))
  signal = np.fft.ifftn(np.fft.ifftshift(signal*gauss_filter))
  normal_sig_abs = np.abs(signal)/np.abs(signal).max()

  # generate data for isosurface of the 3D data 
  fig_data = go.Isosurface( x = xv.flatten(), y = yv.flatten(), z = zv.flatten(),
                           value = normal_sig_abs.flatten(), isomin=0.7, isomax=0.7)

  # generate plots
  clear_output(wait=True) # need this to discard previous figs
  fig = go.Figure( data = fig_data )
  fig.show()
